<a href="https://colab.research.google.com/github/BartoszJanJerzy/stats/blob/main/Boredom_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#WELCOME!

We (MA Bartosz Wójtowicz, PROF Stanisława Steuden, The John Paul II Catholic University of Lublin) wrote this this code to calculate results in our article titled 'Do you feel like a bored stiff? Experimentally Induced Boredom Increases Hostility, Avoidance Motivation and Decreases Mindfulness'.

---

What is the scehdule?

1 - import libraries

2 - functions used in script

3 - import data and screening

4 - reliability coeffs (Cronbach's alpha)

5 - select subscale to analyze

6 - correlation matrix

7 - main analysis: linear regressions

8 - extra analysis: curvilinear regressions

---

We share this code with you, feel free to use or check it.

#1 - Libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import pearsonr

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score

#2 - Functions

In [11]:
def CronbachAlphaa(df_list, names_list):
  subscales = []
  alpha = []

  for df, name in zip(df_list, names_list):
    # 1. Transform the df into a correlation matrix
    df_corr = df.corr()

    # 2.1 Calculate N
    # The number of variables equals the number of columns in the df
    N = df.shape[1]

    # 2.2 Calculate R
    # For this, we'll loop through the columns and append every
    # relevant correlation to an array calles "r_s". Then, we'll
    # calculate the mean of "r_s"
    rs = np.array([])
    for i, col in enumerate(df_corr.columns):
        sum_ = df_corr[col][i+1:].values
        rs = np.append(sum_, rs)
    mean_r = np.mean(rs)

    # 3. Use the formula to calculate Cronbach's Alpha
    cronbach_alpha = (N * mean_r) / (1 + (N - 1) * mean_r)
    cronbach_alpha = round(cronbach_alpha, 3)

    subscales.append(name)
    alpha.append(cronbach_alpha)
  
  alpha_dict = {
      'subscale':subscales,
      'alpha':alpha
  }

  return pd.DataFrame(alpha_dict)

#3 - DATA

In [ ]:
data_raw = pd.read_excel('/content/data.xlsx')
data_whole = data_raw.copy()

emotion_pre_df = pd.read_excel('/content/emotions_pre.xlsx')
emotion_post_df = pd.read_excel('/content/emotions_post.xlsx')
data_whole

In [3]:
for c in data_whole.columns:
  print(c)

id
sex
age
bored
boredom_level
std_boredom_level
photo_1
photo_2
photo_3
photo_4
photo_5
photo_6
photo_7
photo_8
photo_9
photo_10
photo_11
photo_12
rt_1
rt_2
rt_3
rt_4
rt_5
rt_6
rt_7
rt_8
rt_9
rt_10
rt_11
rt_12
neg_aff_1
fear_1
sad_1
guilt_1
hostility_1
disgust_1
tired_1
pos_aff_1
joy_1
self_conf_1
mindfulness_1
calm_1
surprise_1
neg_aff_2
fear_2
sad_2
guilt_2
hostility_2
disgust_2
tired_2
pos_aff_2
joy_2
self_conf_2
mindfulness_2
calm_2
surprise_2
rt_neg_aff
rt_fear
rt_sad
rt_guilt
rt_hostility
rt_disgust
rt_tired
rt_pos_aff
rt_joy
rt_self_conf
rt_mindfulness
rt_calm
rt_surprise
neg_aff
fear
sad
guilt
hostility
disgust
tired
pos_aff
joy
self_conf
mindfulness
calm
surprise


##Screening data
1. Delete records if participant did not get bored
2. Delete outliers -> delete record if results of at least one subscale is lower than -3SD or higher than +3SD

###Prepare filters

In [8]:
mean_boredom = data_whole['boredom_level'].mean()
sd_boredom = data_whole['boredom_level'].std()

filter_1 = data_whole['bored'] == 'yes'
filter_2 = data_whole['boredom_level'] > mean_boredom - 3*sd_boredom
filter_3 = data_whole['boredom_level'] < mean_boredom + 3*sd_boredom

###Use filters

In [9]:
filters = [filter_1, filter_2, filter_3]

for filter in filters:
  data_whole = data_whole[filter]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [10]:
print(f'The research sample - after screening -  has N={len(data_whole)}')

The research sample - after screening -  has N=59


#4 - RELIABILITY COEFFS

In [16]:
for c in emotion_pre_df.columns:
  print(c)

wesoły
zdegustowany
uważny
wstydliwy
ospały
śmiały
zaskoczony
silny
czujący pogardę
odprężony
drażliwy
zachwycony
zainspirowany
nieustraszony
zdegustowany sobą
smutny
wyciszony
obawiajacy się
zmęczony
zdumiony
chwiejny
szczęśliwy
zastraszony
sam
czujny
zaniepokojony
rozłoszczony
odważny
przygnębiony
nieśmiały
aktywny
winny
radosny
nerwowy
samotny
śpiący
podekscytwany
wrogi
dumny
roztrzęsiony
żwawy
zawstydzony
na luzie
przestraszony
senny
zły na siebie
entuzjastyczny
przybity
bojaźliwy
zestresowany
zasługujący na potępienie
zdecydowany
przerażony
zadziwiony
zainteresowany
czujacy odrazę
pewny siebie
energiczny
skoncentrowany
niezadowolony z siebie


##Pre-test

In [41]:
df_pos_pre = emotion_pre_df[['uważny','silny','zainspirowany','czujny','aktywny','podekscytwany','dumny','entuzjastyczny','zdecydowany','zainteresowany']]
df_neg_pre = emotion_pre_df[['drażliwy','obawiajacy się','zaniepokojony','winny','nerwowy','wrogi','roztrzęsiony','zawstydzony','przestraszony','zestresowany']]
df_fear_pre = emotion_pre_df[['obawiajacy się','chwiejny','nerwowy','roztrzęsiony','przestraszony','przerażony']]
df_sad_pre = emotion_pre_df[['smutny','sam','przygnębiony','samotny','przybity']]
df_guilt_pre = emotion_pre_df[['zdegustowany','winny','zawstydzony','zły na siebie','zasługujący na potępienie','niezadowolony z siebie']]
df_host_pre = emotion_pre_df[['zdegustowany','czujący pogardę','drażliwy','rozłoszczony','wrogi','czujacy odrazę']]
df_joy_pre = emotion_pre_df[['wesoły','zachwycony','szczęśliwy','radosny','podekscytwany','żwawy','entuzjastyczny','energiczny']]
df_mind_pre = emotion_pre_df[['uważny','czujny','zdecydowany','skoncentrowany']]
df_conf_pre = emotion_pre_df[['śmiały','silny','nieustraszony','odważny','dumny','pewny siebie']]
df_tired_pre = emotion_pre_df[['ospały','zmęczony','śpiący','senny']]
df_calm_pre = emotion_pre_df[['odprężony','wyciszony','na luzie']]
df_surp_pre = emotion_pre_df[['zaskoczony','zdumiony','zadziwiony']]

df_list_pre = [df_neg_pre, df_fear_pre, df_sad_pre, df_guilt_pre, df_host_pre, df_tired_pre, df_pos_pre, df_joy_pre, df_conf_pre, df_mind_pre, df_calm_pre, df_surp_pre]
names_pre = ['neg_pre','fear_pre','sad_pre','guilt_pre','host_pre','tired_pre','pos_pre','joy_pre','conf_pre','mind_pre','calm_pre','surp_pre']

In [43]:
df_alpha_pre = CronbachAlphaa(df_list_pre, names_pre)

df_alpha_pre

,subscale,alpha
0,neg_pre,0.849
1,fear_pre,0.798
2,sad_pre,0.875
3,guilt_pre,0.808
4,host_pre,0.725
5,tired_pre,0.903
6,pos_pre,0.716
7,joy_pre,0.806
8,conf_pre,0.816
9,mind_pre,0.633


##Post-test

In [39]:
df_pos_post = emotion_post_df[['uważny','silny','zainspirowany','czujny','aktywny','podekscytwany','dumny','entuzjastyczny','zdecydowany','zainteresowany']]
df_neg_post = emotion_post_df[['drażliwy','obawiający się','zaniepokojony','winny','nerwowy','wrogi','roztrzęsiony','zawstydzony','przestraszony','zestresowany']]
df_fear_post = emotion_post_df[['obawiający się','chwiejny','nerwowy','roztrzęsiony','przestraszony','przerażony']]
df_sad_post = emotion_post_df[['smutny','sam','przygnębiony','samotny','przybity']]
df_guilt_post = emotion_post_df[['zdegustowany','winny','zawstydzony','zły na siebie','zasługujący na potępienie','niezadowolony z siebie']]
df_host_post = emotion_post_df[['zdegustowany','czujący pogardę','drażliwy','rozzłoszczony','wrogi','czujący odrazę']]
df_joy_post = emotion_post_df[['wesoły','zachwycony','szczęśliwy','radosny','podekscytwany','żwawy','entuzjastyczny','energiczny']]
df_mind_post = emotion_post_df[['uważny','czujny','zdecydowany','skoncentrowany']]
df_conf_post = emotion_post_df[['śmiały','silny','nieustraszony','odważny','dumny','pewny siebie']]
df_tired_post = emotion_post_df[['ospały','zmęczony','śpiący','senny']]
df_calm_post = emotion_post_df[['odprężony','wyciszony','na luzie']]
df_surp_post = emotion_post_df[['zaskoczony','zdumiony','zadziwiony']]

df_list_post = [df_neg_post, df_fear_post, df_sad_post, df_guilt_post, df_host_post, df_tired_post, df_pos_post, df_joy_post, df_conf_post, df_mind_post, df_calm_post, df_surp_post]
names_post = ['neg_post','fear_post','sad_post','guilt_post','host_post','tired_post','pos_post','joy_post','conf_post','mind_post','calm_post','surp_post']

In [40]:
df_alpha_post = CronbachAlphaa(df_list_post, names_post)

df_alpha_post

,subscale,alpha
0,neg_post,0.883
1,fear_post,0.879
2,sad_post,0.884
3,guilt_post,0.758
4,host_post,0.890
5,tired_post,0.861
6,pos_post,0.846
7,joy_post,0.841
8,conf_post,0.758
9,mind_post,0.765


##General

In [47]:
dict_alpha = {
    'subscale':names_pre+names_post,
    'alpha':df_alpha_pre['alpha'].to_list()+df_alpha_post['alpha'].to_list()
}

df_alpha = pd.DataFrame(dict_alpha).T
df_alpha

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
subscale,neg_pre,fear_pre,sad_pre,guilt_pre,host_pre,tired_pre,pos_pre,joy_pre,conf_pre,mind_pre,calm_pre,surp_pre,neg_post,fear_post,sad_post,guilt_post,host_post,tired_post,pos_post,joy_post,conf_post,mind_post,calm_post,surp_post
alpha,0.849,0.798,0.875,0.808,0.725,0.903,0.716,0.806,0.816,0.633,0.439,0.747,0.883,0.879,0.884,0.758,0.89,0.861,0.846,0.841,0.758,0.765,0.506,0.816
